## Problem 1: Join accessibility datasets into a grid and visualise the data on a map (10 points)

![P1 example result](images/travel_times_example_720x720px.png)

*Sample result: Travel times by public transport (PT) and car to Itis and Myyrmanni*

Your task in *problem 1* is to combine data from non-spatial data sets (travel times between places)
and a spatial data set (grid cells that represent the places), and plot the combined data set to
visualise the travel times to two shopping centres from every other place in the Helsinki metropolitan
area. 

In particular, this task comprises of three major steps:

1. Read the grid cell data set
2. Read a travel time data set,
    - discard unnecessary columns,
    - rename the relevant columns to include a reference to the respective shopping centre,
    - join the relevant columns to the grid data set
3. Classify the travel times for both travel modes (public transport and private car) into five-minute intervals 

Repeat the second step for each of the two shopping centres (Itis, Myyrmanni).


---

### a) Read the grid cell data set

The grid cells are derived from the ‘YKR’ data set, that is [published by the Finnish Environmental Institute
(SYKE)](https://www.ymparisto.fi/fi-fi/elinymparisto_ja_kaavoitus/yhdyskuntarakenne/tietoa_yhdyskuntarakenteesta/yhdyskuntarakenteen_seurannan_aineistot)
and collects a variety of indicators relating to the social and built-up structure of the country. In
an effort to harmonise different data products of other institutions, the YKR grid cell data set has
become a reference for many data products, including, for instance, the [travel time data set produced
at the Digital Geography Lab](https://blogs.helsinki.fi/accessibility/helsinki-region-travel-time-matrix/).

You can find the YKR data set in the directory `data` in *GeoPackage* format: `YKR_grid_EPSG3067.gpkg`.
It contains a polygon geometry column, and an (integer) identifier: `YKR_ID`.

Load the data into a `geopandas.GeoDataFrame` and call it `grid`.

In [ ]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

In [2]:
# ADD YOUR OWN CODE HERE
import geopandas as gpd
grid = gpd.read_file(r'C:\Users\IP500\Documents\GitHub\Automating-GIS-Processes\Exercise 4\data\YKR_grid_EPSG3067.gpkg')


---

### b) Read the travel time data sets and join them to the grid cells

Inside the `data` directory, you will find a set of semicolon-separated text files with travel times 
to each of a set of shopping centres in the Helsinki region (*this exercise was conceived before
REDI and Tripla started operation*).

The individual files have file names following the schema `travel_times_to_[XXXXX]_[Shopping_Centre]`, where
`[Shopping_Centre]` is the name of one of the seven shopping centres included in the data set, and `[XXXXX]`,
coincidently, referring to the YKR grid cell in which the shopping centre is located (although you should not
need to use this ID in this exercise).

The data sets contain computed travel times between different places in the metropolitan area. In order to 
produce such a *travel time matrix*, all connections from all origins to all destinations are calculated,
and then recorded in a table. 

#### Columns

The data sets we use have many **columns**, but only a few are interesting for this task:

- `from_id`: the `YKR_ID` of the *origin* grid cell
- `to_id`: refers to the `YKR_ID` of the *destination* grid cell (here: the one containing the shopping centre).
- `pt_r_t`: how long does it take to travel from `from_id` to `to_id`, in minutes, using public transport?
- `car_r_t`: how long does it take to drive a car from `from_id` to `to_id`, in minutes?

Note that `from_id` and `to_id` relate to the YKR `grid` data set’s `YKR_ID` column. Each input data set has only
one unique `to_id`, as the data has been split up to relate to one destination (a shopping centre), only, but many
unique values for `from_id`, as it covers the travel times from anywhere in the metropolitan area.

#### No-data values

The travel time data set contains some origin-destination pairs (*O/D-pairs*) for which it could not find a
public transport connection, or which are not accessible by car. Such **no-data values** are saves as 
`-1` minutes travel time. Use the [`pandas.Series.replace()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html)
method to replace `-1` with `numpy.nan` to indicate that these cells do not contain valid data.


**IMPORTANT**: While we are having this course, a bug (https://github.com/pandas-dev/pandas/issues/45725)
prevents `pandas.Series.replace()` from working as expected: the line `travel_times["car_r_t"] = travel_times["car_r_t"].replace(-1, numpy.nan)`
fails with a `RecursionError`. There is a workaround: using an alternative syntax, using a `dict` of *before*
and *after* values, similar to how `pandas.DataFrame.rename()` works, does not trigger the issue. 
`travel_times["car_r_t"] = travel_times["car_r_t"].replace({-1: numpy.nan})` works.

---

Read the data sets for the shopping centres ‘Itis’ and ‘Myyrmanni’, discard irrelevant columns, rename the
`pt_r_t` and `car_r_t` columns to include a reference to the shopping centre (e.g., into `pt_r_t_Itis`), and join
the renamed columns to the `grid` data frame. Don’t forget to replace no-data values (`-1`) with `None`.

In [11]:
# ADD YOUR OWN CODE HERE
import pandas as pd
Itis = pd.read_csv(r'C:\Users\IP500\Documents\GitHub\Automating-GIS-Processes\Exercise 4\data\travel_times_to_5944003_Itis.txt', sep = ';')
Myyrmanni = pd.read_csv(r'C:\Users\IP500\Documents\GitHub\Automating-GIS-Processes\Exercise 4\data\travel_times_to_5902043_Myyrmanni.txt', sep = ';')

In [12]:
import numpy 
Itis["car_r_t"] = Itis["car_r_t"].replace({-1: numpy.nan})
Myyrmanni["car_r_t"] = Myyrmanni["car_r_t"].replace({-1: numpy.nan})

In [26]:
#Itis['pt_r_t'].rename(Itis['pt_r_t_Itis'])
#Itis['car_r_t'] = Itis['car_r_t_Itis']
Itis.rename(columns = {'pt_r_t':'pt_r_t_Itis','car_r_t':'car_r_t_Itis'})
Myyrmanni.rename(columns = {'pt_r_t':'pt_r_t_Myyrmanni','car_r_t':'car_r_t_Myyrmanni'})



,from_id,to_id,walk_t,walk_d,car_r_t_Myyrmanni,car_r_d,car_m_t,car_m_d,pt_r_t_Myyrmanni,pt_r_tt,pt_r_d,pt_m_t,pt_m_tt,pt_m_d
0,5785640,5902043,295,20668,39.0,22321,35,22321,90,121,21119,98,129,21119
1,5785641,5902043,258,18052,39.0,22532,35,22532,93,115,21325,101,127,21325
2,5785642,5902043,259,18131,47.0,22216,42,20440,95,115,21443,103,128,21443
3,5785643,5902043,263,18424,51.0,24773,46,22996,99,126,21736,107,132,21736
4,5787544,5902043,288,20179,38.0,21895,34,21895,83,103,20630,91,108,20630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,6016698,5902043,362,25329,54.0,31686,47,31635,95,111,32743,92,112,31996
13227,6016699,5902043,364,25456,56.0,31373,49,31322,97,111,32870,94,112,32123
13228,6018252,-1,-1,-1,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1
13229,6018253,5902043,367,25686,55.0,31863,48,31812,100,111,33100,97,112,32353


In [31]:
itis = Itis[['from_id','to_id','pt_r_t','car_r_t']]
myyrmanni = Myyrmanni[['from_id','to_id','pt_r_t','car_r_t']]

In [35]:
grid = grid.join(Itis, on = 'from_id', lsuffix='_caller', rsuffix='_other')
#grid= grid.join(Myyrmanni)

In [37]:
grid = grid.join(Myyrmanni, on = 'from_id_caller', lsuffix='_callerm', rsuffix='_otherm')

In [13]:
# NON-EDITABLE TEST CELL
import geopandas
assert type(grid) == geopandas.geodataframe.GeoDataFrame, "Output should be a geodataframe."

In [38]:
# NON-EDITABLE TEST CELL
# Check that the merged output have (at least) the necessary columns
required_columns = ['pt_r_t_Itis', 'car_r_t_Itis', 'pt_r_t_Myyrmanni', 'car_r_t_Myyrmanni', 'geometry']

assert all(column in grid.columns for column in required_columns), "Couldn’t find all required columns."

AssertionError: Couldn’t find all required columns.

In [39]:
# NON-EDITABLE TEST CELL
# Check that -1 values are not present in the columns
for shopping_centre in ("Itis", "Myyrmanni"):
    for column in ("car_r_t", "pt_r_t"):
        assert -1 not in grid[f"{column}_{shopping_centre}"], "NoData values (-1) should be removed from the data!"

KeyError: 'car_r_t_Itis'


---

### c) Classify the travel times into five-minute intervals 

Classify the travel times for both travel modes (public transport and private car) into five-minute intervals 
- Create a **[`mapclassify.UserDefined` classifier](https://github.com/pysal/mapclassify#userdefined)**,
  using these class breaks: `5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60`
- Apply the classifier to `pt_r_t_[ShoppingCentre]` and `car_r_t_[ShoppingCentre]` columns,
  and store the results to `pt_r_t_cl_[ShoppingCentre]` and `car_r_t_cl_[ShoppingCentre]` columns, respectively

*Note that these 12 class breaks will produce 13 classes (1 class break divides the data into two classes)*

In [ ]:
# ADD YOUR OWN CODE HERE
import mapclassify


In [ ]:
# NON-EDITABLE TEST CELL
# Check the output
print("travel times by public transport:")
grid[['pt_r_t_Itis', 'pt_r_t_cl_Itis']].head()

In [ ]:
# NON-EDITABLE TEST CELL
# Check the output
print("Travel times by car:")
grid[["car_r_t_Myyrmanni", "car_r_t_cl_Myyrmanni"]].head()

### d) Plot the classified travel times

Create a 2⨉2 plot, in which you plot the *classified* travel times by public transport (left) and by car (right),
to Itis (top) and Myyrmanni (bottom):

- [Create subplots using `matplotlib.pyplot.subplots()`](https://geo-python-site.readthedocs.io/en/latest/notebooks/L7/advanced-plotting.html#using-subplots),
  with 2 rows and 2 columns. Increase the figure size to 10⨉10"
- Add appropriate titles for the subplots
- Save the figure to a file called `shopping_centre_accessibility.png` in the `DATA_DIRECTORY`


In [ ]:
# ADD YOUR OWN CODE HERE


In [ ]:
# NON-EDITABLE TEST CELL
# Check that output figure file exists
assert (DATA_DIRECTORY / "shopping_centre_accessibility.png").exists()


---

**Finally**, check that you have provided adequate comments in your code.

---

### Reflections

If you want to ask feedback on spesific parts of your code, please leave your comment here


---

### Done!

**Remember to commit the output figure** to your exercise repository!

That’s! Once you have plotted the data and saved the output, you are ready to proceed to *problem 2*.